In [14]:
import numpy as np
import requests
import json
import xmltodict
import re 
import string 
table = str.maketrans('', '', string.ascii_lowercase)

In [9]:
def Car_info(text):
    vehicle_reg_no = text.strip() #insert the correct registration number
    username = "meemo1" #insert your user name
    url = "http://www.regcheck.org.uk/api/reg.asmx/CheckIndia?RegistrationNumber=" + vehicle_reg_no + "&username="+username
    url=url.replace(" ","%20")
    r = requests.get(url)
    n = xmltodict.parse(r.content)
    k = json.dumps(n)
    df = json.loads(k)
    l=df["Vehicle"]["vehicleJson"]
    p=json.loads(l)
    s="Your car's details are:\n"+"Owner name: "+str(p['Owner'])+"\n"+"Car Company: "+str(p['CarMake']['CurrentTextValue'])+"\n"+"Car Model: "+str(p['CarModel']['CurrentTextValue'])+"\n"+"Fuel Type: "+str(p['FuelType']['CurrentTextValue'])+"\n"+"Registration Year: "+str(p['RegistrationYear'])+"\n"+"Insurance: "+str(p['Insurance'])+"\n"+"Vehicle ID: "+str(p['VechileIdentificationNumber'])+"\n"+"Engine No.: "+str(p['EngineNumber'])+"\n"+"Location RTO: "+str(p['Location'])
    print(s)

In [10]:
Car_info("UP32CB0211")

Your car's details are:
Owner name: 
Car Company: MARUTI
Car Model: WAGON R LXI (1061CC)
Fuel Type: 
Registration Year: 2007
Insurance: 01/01/0001
Vehicle ID: 
Engine No.: 
Location RTO: UTTAR PRADESH-LUCKNOW


In [4]:
import streamlit as st
import os
from PIL import Image
import shutil
import requests
import json
import xmltodict
import glob
import re
import cv2
import easyocr
import imutils

In [11]:
image_file_path = '../exp/crops/Number_Plate/video.jpg'
image = cv2.imread(image_file_path)
image = imutils.resize(image, width=300 )
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray_image = cv2.bilateralFilter(gray_image, 11, 17, 17) 
edged = cv2.Canny(gray_image, 30, 200) 
cnts,new = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True) [:30]
screenCnt = None

os.makedirs('./final_plate/',exist_ok=True)
i=7
for c in cnts:
    perimeter = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * perimeter, True)
    if len(approx) == 4: 
        screenCnt = approx
        x,y,w,h = cv2.boundingRect(c) 
        new_img=image[y:y+h,x:x+w]
        print(x,y,w,h)
        os.chdir('./final_plate/')
        cv2.imwrite(str(i)+'.png',new_img)
        os.chdir('../')
        i+=1
        break

licensePlate1 = image[y:y+h, x:x + w]
licensePlate2=image[y:y + h, x+15:x + w]

if len(os.listdir('./final_plate'))==0:
    os.chdir('./final_plate')
    cv2.imwrite('cropped1.png',licensePlate1)
    cv2.imwrite('cropped2.png',licensePlate2)
    os.chdir("../")
else:
    files = glob.glob('./fimal_plate')
    for f in files:
            os.remove(f)
    os.chdir('./final_plate')
    cv2.imwrite('cropped1.png',licensePlate1)
    cv2.imwrite('cropped2.png',licensePlate2)
    os.chdir("../")


reader = easyocr.Reader(['en'])
img1=os.path.join('./final_plate',"cropped1.png")
img2=os.path.join('./final_plate',"cropped2.png")


image1 = cv2.imread(img1)

img1_res1= reader.readtext(image1)
print(img1_res1)

image2 = cv2.imread(img2)
img2_res2= reader.readtext(image2)

if len(img2_res2)>1:
    res1=str("".join(re.split("[^a-zA-Z0-9]*", img1_res1[1][1]))).upper()
    res2=str("".join(re.split("[^a-zA-Z0-9]*", img2_res2[1][1]))).upper()
    res=[res1,res2]
    res
else:
    res1=str("".join(re.split("[^a-zA-Z0-9]*", img1_res1[0][1]))).upper()
    res2=str("".join(re.split("[^a-zA-Z0-9]*", img2_res2[0][1]))).upper()
    res=[res1,res2]
    res

digit2=res[1][-4:]
to_remove_chars = {"O": "0", "I": "1", "G":"6","S": "6", "Z": "7","4":"A"}
for char in to_remove_chars.keys():
    digit2 = digit2.replace(char, to_remove_chars[char])


st1=res[1][:2]
to_remove_chars = {"1": "I", "5": "S", "6":"G", "7": "Z", "0": "O","4":"A"}
for char in to_remove_chars.keys():
    st1 = st1.replace(char, to_remove_chars[char])


digit1=res[1][2:4]
to_remove_chars = {"I": "1","G":"6", "S": "5", "Z": "7", "O": "0","A":"4"}
for char in to_remove_chars.keys():
    digit1 = digit1.replace(char, to_remove_chars[char])


st2=res[1][4:6]
to_remove_chars = {"1": "I", "6":"G", "5": "S", "7": "Z", "0": "O","4":"A"}
for char in to_remove_chars.keys():
    st2 = st2.replace(char, to_remove_chars[char])


vehicle_reg_no=st1+digit1+st2+digit2

2022-12-23 20:26:59.412 WARNING easyocr.easyocr: CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[0, 0], [235, 0], [235, 54], [0, 54]], '4BZOBc6 175', 0.1848509542422097)]


In [12]:
vehicle_reg_no

'BZ0BCG6177'